In [14]:
import requests as re
import pandas as pd
import json
import os
import sqlite3 as sql

# Load API Keys from OS environmental variables
fs_client_id = os.environ["FOURSQUARE_CLIENT_ID"]
fs_client_secret = os.environ["FOURSQUARE_CLIENT_SECRET"]
y_API_key = os.environ["YELP_API_KEY"]

# Define number of venues to index
num_venues = 15

# Define API URLs with corresponding parametrs 
fs_url = 'https://api.foursquare.com/v2/venues/explore'
fs_params = {'client_id' : fs_client_id, 'client_secret' : fs_client_secret, 'll' : '43.6491,-79.3796',
            'radius' : 5000, 'section' : 'food', 'v' : '20210114', 'limit' : num_venues}
y_url = 'https://api.yelp.com/v3/businesses/search'
y_params = {'term' : 'food', 'latitude' : 43.6491, 'longitude' : -79.3796, 'radius' : 5000 , 
            'limit' : num_venues, 'sort_by' : 'rating', 'attributes' : 'hot_and_new'}
y_headers = {'Authorization' : f'Bearer {y_API_key}'}

# Make API calls
fs_response = re.get(fs_url, params=fs_params).json()
y_response = re.get(y_url, params=y_params, headers=y_headers).json()


In [2]:
fs_venue_id_list = []
fs_venue_name_list = []
fs_venue_address_list = []
for i in range(0,num_venues):
    fs_venue_id_list.append(fs_response['response']['groups'][0]['items'][i]['venue']['id'])
    fs_venue_name_list.append(fs_response['response']['groups'][0]['items'][i]['venue']['name'])
    fs_venue_address_list.append(fs_response['response']['groups'][0]['items'][i]['venue']['location']['formattedAddress'][0])
fs_venues_dict = {'name': fs_venue_name_list, 'address' : fs_venue_address_list}

In [3]:
ratings_list = []
for value in fs_venue_id_list:
    try:
        VENUE_ID = value
        fs_premium_url = f'https://api.foursquare.com/v2/venues/{VENUE_ID}'
        fs_premium_params = {'client_id' : fs_client_id, 'client_secret' : fs_client_secret,'v':'20200114'}
        premium_response = re.get(fs_premium_url, params=fs_premium_params).json()
        rating = premium_response['response']['venue']['rating']
        ratings_list.append(rating)
    except KeyError:
        ratings_list.append('No Rating Yet.')

In [4]:
foursquare_df = pd.DataFrame(fs_venues_dict)
foursquare_df['rating'] = ratings_list
foursquare_df

,name,address,rating
0,Richmond Station,1 Richmond Street West (Yonge Street),No Rating Yet.
1,Dineen Coffee,140 Yonge St (at Temperance St),No Rating Yet.
2,Mos Mos Coffee,Commerce Court East,No Rating Yet.
3,Hogtown Smoke,1959 Queen St E,No Rating Yet.
4,Canoe,66 Wellington St West (at Bay Street),No Rating Yet.
5,Pai,18 Duncan St (Adelaide and Duncan),No Rating Yet.
6,The Keg Steakhouse + Bar - Esplanade,26 The Esplanade,No Rating Yet.
7,Mystic Muffin,113 Jarvis St (at Richmond St E),No Rating Yet.
8,GEORGE Restaurant,111c Queen St East (Church),No Rating Yet.
9,Byblos Toronto,11 Duncan Street,No Rating Yet.


In [5]:
y_response = re.get(y_url, params=y_params, headers=y_headers).json()

In [6]:
yelp_df = pd.DataFrame(y_response['businesses'])

In [7]:
yelp_address_list = []
for i in range(len(y_response['businesses'])):
    yelp_address_list.append(y_response['businesses'][i]['location']['address1'])

In [8]:
yelp_df['address'] = yelp_address_list
yelp_df['rating'] = yelp_df['rating'] * 2

In [9]:
yelp_df = yelp_df[['name','address','rating']] 

In [10]:
yelp_df

,name,address,rating
0,J San Sushi Bar,186 Jarvis Street,10.0
1,Gurume Sushi,45 Carlton Street,10.0
2,Dough T O,719 Queen Street W,8.0
3,Mama Lee's Korean Kitchen,16 Yonge Street,9.0
4,Bake Code Croissanterie,626 Yonge St,10.0
5,Minami Toronto,225 King Street W,9.0
6,Matty’s Patty’s Burger Club,923a Queen St W,6.0
7,Birria Catrina,214 Augusta Avenue,8.0
8,Tapi Go!,835 Bloor Street W,9.0
9,Courage Cookies,28 Bathurst Street,10.0


In [11]:
master_df = pd.concat([yelp_df,foursquare_df])
master_df

,name,address,rating
0,J San Sushi Bar,186 Jarvis Street,10
1,Gurume Sushi,45 Carlton Street,10
2,Dough T O,719 Queen Street W,8
3,Mama Lee's Korean Kitchen,16 Yonge Street,9
4,Bake Code Croissanterie,626 Yonge St,10
5,Minami Toronto,225 King Street W,9
6,Matty’s Patty’s Burger Club,923a Queen St W,6
7,Birria Catrina,214 Augusta Avenue,8
8,Tapi Go!,835 Bloor Street W,9
9,Courage Cookies,28 Bathurst Street,10


In [15]:
connection = sql.connect('Trending_Venues_Database.db')
master_df.to_sql('Venues',connection, if_exists='replace',index=False)

In [16]:
pd.read_sql('SELECT * from venues', connection)

,name,address,rating
0,J San Sushi Bar,186 Jarvis Street,10.0
1,Gurume Sushi,45 Carlton Street,10.0
2,Dough T O,719 Queen Street W,8.0
3,Mama Lee's Korean Kitchen,16 Yonge Street,9.0
4,Bake Code Croissanterie,626 Yonge St,10.0
5,Minami Toronto,225 King Street W,9.0
6,Matty’s Patty’s Burger Club,923a Queen St W,6.0
7,Birria Catrina,214 Augusta Avenue,8.0
8,Tapi Go!,835 Bloor Street W,9.0
9,Courage Cookies,28 Bathurst Street,10.0


In [21]:
master_df.sample(n=3)

,name,address,rating
6,The Keg Steakhouse + Bar - Esplanade,26 The Esplanade,No Rating Yet.
12,Crepe TO,52 Church Street,No Rating Yet.
10,Xing Fu Tang,316 College Street,8
